In [2]:
from pyspark.streaming.kafka import KafkaUtils
sc = SparkContext(appName="PythonInclassStreaming")
ssc = StreamingContext(sc, 5)

In [3]:
# Receiver
kvs = KafkaUtils.createStream(ssc, "ip-20-0-21-161.ec2.internal:2181", "spark-streaming-consumer", {"ujjwal_data": 1})
#kvs = KafkaUtils.createStream(ssc, "localhost:2181", "spark-streaming-consumer", {"end_to_end": 1})

# Dstream
lines = kvs.map(lambda x: x[1])

In [4]:
def process(t, rdd):
    if rdd.isEmpty():
        print("Empty Rdd")
        return
    
    spark=SparkSession.builder.getOrCreate()
    rawRdd = rdd.map(lambda x: x.split(',')).map(lambda d: Row(NAME_CONTRACT_TYPE=d[0], 
                                                               CODE_GENDER=d[1], 
                                                               FLAG_OWN_CAR=d[2], 
                                                               FLAG_OWN_REALTY=d[3], 
                                                               CNT_CHILDREN=int(d[4]), 
                                                               AMT_INCOME_TOTAL=float(d[5]), 
                                                               AMT_CREDIT=float(d[6]), 
                                                               AMT_ANNUITY=float(d[7]), 
                                                               NAME_INCOME_TYPE=d[8], 
                                                               NAME_EDUCATION_TYPE=d[9],
                                                               NAME_FAMILY_STATUS=d[10], 
                                                               NAME_HOUSING_TYPE=d[11], 
                                                               DAYS_BIRTH=int(d[12]), 
                                                               DAYS_EMPLOYED=int(d[13]), 
                                                               FLAG_MOBIL=int(d[14]), 
                                                               FLAG_EMP_PHONE=int(d[15]), 
                                                               FLAG_WORK_PHONE=int(d[16]), 
                                                               FLAG_CONT_MOBILE=int(d[17]), 
                                                               FLAG_PHONE=int(d[18]), 
                                                               CNT_FAM_MEMBERS=float(d[19]), 
                                                               REGION_RATING_CLIENT=int(d[20]), 
                                                               REGION_RATING_CLIENT_W_CITY=int(d[21]), 
                                                               REG_REGION_NOT_LIVE_REGION=int(d[22]), 
                                                               REG_REGION_NOT_WORK_REGION=int(d[23]), 
                                                               ORGANIZATION_TYPE=d[24], 
                                                               FLAG_DOCUMENT_2=int(d[25]), 
                                                               FLAG_DOCUMENT_3=int(d[26]), 
                                                               FLAG_DOCUMENT_4=int(d[27]), 
                                                               FLAG_DOCUMENT_5=int(d[28]), 
                                                               FLAG_DOCUMENT_6=int(d[29]), 
                                                               FLAG_DOCUMENT_7=int(d[30]), 
                                                               FLAG_DOCUMENT_8=int(d[31]), 
                                                               FLAG_DOCUMENT_9=int(d[32]), 
                                                               FLAG_DOCUMENT_10=int(d[33]), 
                                                               FLAG_DOCUMENT_11=int(d[34]), 
                                                               FLAG_DOCUMENT_12=int(d[35])))
    raw = spark.createDataFrame(rawRdd)
    print(raw.show(5))
    if not rdd.isEmpty():
        df = rdd.map(lambda r:Row(message=r))
        df = df.withColumn('CREDIT_INCOME_PERCENT',F.col('AMT_CREDIT')/F.col('AMT_INCOME_TOTAL'))
        df = df.withColumn("AGE", F.col("DAYS_BIRTH")/-365)
        df = df.withColumn('DAYS_EMPLOYED_ANOM',F.col("DAYS_EMPLOYED") == 365243)
        df = df.withColumn('DAYS_EMPLOYED', F.when(F.col('DAYS_EMPLOYED') == 365243, 0).otherwise(F.col('DAYS_EMPLOYED')))
        df = df.withColumn('CREDIT_INCOME_PERCENT',F.col('AMT_CREDIT')/F.col('AMT_INCOME_TOTAL'))
        df = df.withColumn('ANNUITY_INCOME_PERCENT',F.col('AMT_ANNUITY')/F.col('AMT_INCOME_TOTAL'))
        df = df.withColumn('CREDIT_TERM',F.col('AMT_ANNUITY')/F.col('AMT_CREDIT'))
        df = df.withColumn('DAYS_EMPLOYED_PERCENT',F.col('DAYS_EMPLOYED')/F.col('DAYS_BIRTH'))
        pipeline = PipelineModel.load('hdfs:///user/edureka_1033463/pymodellr_ujjwal')
        predictions = pipeline.transform(df)
        print(predictions.select('prediction').show())

In [5]:
lines.pprint()
lines.foreachRDD(process)

In [6]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2020-06-13 05:57:55
-------------------------------------------

Empty Rdd
-------------------------------------------
Time: 2020-06-13 05:58:00
-------------------------------------------

Empty Rdd
-------------------------------------------
Time: 2020-06-13 05:58:05
-------------------------------------------
Cash,M,N,Y,0,202500.0,406597.5,24700.5,Working,Secondary,Single,House,-9461,-637,1,1,0,1,1,1.0,2,2,0,0,Business Entity Type 3,0,1,0,0,0,0,0,0,0,0,0

+-----------+----------+----------------+------------+---------------+-----------+----------+-------------+----------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+--------------+----------+------------+---------------+----------+---------------+------------------+-------------------+------------------+-----------------+----------------+---------

Py4JJavaError: An error occurred while calling o48.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/opt/cloudera/parcels/SPARK2/lib/spark2/python/pyspark/streaming/util.py", line 65, in call
    r = self.func(t, *rdds)
  File "<ipython-input-4-2b99f1855cb2>", line 47, in process
    df = df.withColumn('CREDIT_INCOME_PERCENT',F.col('AMT_CREDIT')/F.col('AMT_INCOME_TOTAL'))
AttributeError: 'PipelinedRDD' object has no attribute 'withColumn'

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.apache.spark.streaming.api.python.TransformFunction.apply(PythonDStream.scala:78)
	at org.apache.spark.streaming.api.python.PythonDStream$$anonfun$callForeachRDD$1.apply(PythonDStream.scala:179)
	at org.apache.spark.streaming.api.python.PythonDStream$$anonfun$callForeachRDD$1.apply(PythonDStream.scala:179)
	at org.apache.spark.streaming.dstream.ForEach

-------------------------------------------
Time: 2020-06-13 05:58:20
-------------------------------------------

Empty Rdd
-------------------------------------------
Time: 2020-06-13 05:58:25
-------------------------------------------

Empty Rdd
-------------------------------------------
Time: 2020-06-13 05:58:30
-------------------------------------------

Empty Rdd
-------------------------------------------
Time: 2020-06-13 05:58:35
-------------------------------------------

Empty Rdd
-------------------------------------------
Time: 2020-06-13 05:58:40
-------------------------------------------

Empty Rdd
-------------------------------------------
Time: 2020-06-13 05:58:45
-------------------------------------------

Empty Rdd
-------------------------------------------
Time: 2020-06-13 05:58:50
-------------------------------------------

Empty Rdd


In [7]:
ssc.stop()